In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [30]:
import json

In [3]:
spark

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [4]:
rdd = sc.textFile('/labs/laba01/ml-100k/u.data').map(lambda x: x.split('\t'))

###user id | item id | rating | timestamp. 

In [10]:
rdd.count()

100000

In [11]:
(rdd.filter(lambda line:line[1]=="22")
 .map(lambda line: (line[0],line[1], line[2])).count()
)


297

In [21]:
rate_film = []
for i in range(1,6):
    
    x = (rdd.filter(lambda line: (line[1]=="22")&(line[2]==f'{i}'))
     .map(lambda line: (line[0],line[1], line[2])).count()
    )
    rate_film.append(x)



In [27]:
print(rate_film)
s = 0
for i in rate_film:
    s+=i
    
print(s)

[5, 14, 46, 98, 134]
297


In [24]:
rate_all = []
for i in range(1,6):
    
    x = (rdd.filter(lambda line: line[2]==f'{i}')
     .map(lambda line: (line[0],line[1], line[2])).count()
    )
    rate_all.append(x)

In [28]:
print(rate_all)
s = 0
for i in rate_all:
    s+=i
    
print(s)

[6110, 11370, 27145, 34174, 21201]
100000


In [29]:
ans = {
    'hist_film': rate_film,
    'hist_all': rate_all
}

In [31]:
jsonString = json.dumps(ans, indent=4)
print(jsonString)

{
    "hist_film": [
        5,
        14,
        46,
        98,
        134
    ],
    "hist_all": [
        6110,
        11370,
        27145,
        34174,
        21201
    ]
}


In [32]:
jsonFile = open("lab01.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [ ]:
spark.stop()